# Collecte et Comparaison des Données Ethereum

Ce notebook a pour objectif de comparer les données de transactions Ethereum obtenues via deux sources différentes :
1. L'API Etherscan : pour obtenir l'historique des transactions d'une adresse spécifique
2. Web3 : pour obtenir les transactions des blocs récents

Cette comparaison nous permettra de :
- Valider la cohérence des données entre les sources
- Enrichir notre jeu de données avec des informations complémentaires
- Identifier les avantages et limitations de chaque source

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
from pathlib import Path

# Add src to PYTHONPATH
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.config import RAW_DATA_DIR, PROCESSED_DATA_DIR
from src.data_collection import fetch_transactions, fetch_address_transactions_etherscan

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

## 1. Collecte des données via Etherscan

Nous allons d'abord collecter l'historique des transactions pour une adresse spécifique via l'API Etherscan. 
Pour cet exemple, nous utiliserons l'adresse du hot wallet de Binance : `0x28C6c06298d514Db089934071355E5743bf21d60`

In [3]:
# Adresse du hot wallet de Binance
address = '0x28C6c06298d514Db089934071355E5743bf21d60'

# Collecte des transactions via Etherscan
etherscan_transactions = fetch_address_transactions_etherscan(address)

print(f"Nombre de transactions collectées via Etherscan : {len(etherscan_transactions)}")
etherscan_transactions.head()

2025-01-15 20:41:14,186 - src.etherscan_api - INFO - Retrieved 10000 transactions for address 0x28C6c06298d514Db089934071355E5743bf21d60
2025-01-15 20:41:14,187 - src.data_collection - INFO - Retrieved 10000 transactions for address 0x28C6c06298d514Db089934071355E5743bf21d60
2025-01-15 20:41:14,649 - src.data_collection - INFO - Saved transactions to /Users/guillaumeverbiguie/Desktop/Anomalie_eth/data/raw/0x28C6c06298d514Db089934071355E5743bf21d60_transactions.csv


Nombre de transactions collectées via Etherscan : 10000


,timestamp,block_number,hash,from_address,to_address,value,gas,gas_price
0,2025-01-15 19:40:59,21632037,0x56ff8afe0c24f62b556ce64452e206aaf655f67d1bd3...,0x28c6c06298d514db089934071355e5743bf21d60,0x220522658c89b9adfab5eb14fbc2d7d8fb4a5a54,0.431318,207128,16200807180
1,2025-01-15 19:40:59,21632037,0x789a7b060b466c12733e3a156e111abe63b7f84851ac...,0x28c6c06298d514db089934071355e5743bf21d60,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,0.000000,414256,16200807180
2,2025-01-15 19:40:35,21632035,0xc812371960d437e48d1300ad646de64c748c50d482e9...,0x28c6c06298d514db089934071355e5743bf21d60,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.000000,207128,16253655847
3,2025-01-15 19:40:35,21632035,0x4255a7b71016f147666933981b83c5490c0dc3a25956...,0x28c6c06298d514db089934071355e5743bf21d60,0xed04915c23f00a313a544955524eb7dbd823143d,0.000000,207128,16253655847
4,2025-01-15 19:40:35,21632035,0x656c2e5c5ed05972c922c235dc4ebe3b3b70af699ae9...,0x28c6c06298d514db089934071355e5743bf21d60,0x8b532d1ab0e0c3c98605169644e3e091e4b0ee6f,0.019779,207128,16253655847


## 2. Collecte des données via Web3

Maintenant, nous allons collecter les transactions des blocs récents via Web3 pour comparer avec les données d'Etherscan.

In [4]:
# Collecte des transactions via Web3
web3_transactions = fetch_transactions()

print(f"Nombre de transactions collectées via Web3 : {len(web3_transactions)}")
web3_transactions.head()

Fetching blocks: 100%|██████████| 100/100 [00:39<00:00,  2.51it/s]
2025-01-15 20:42:00,432 - src.data_collection - INFO - Data collection completed. Transactions saved to /Users/guillaumeverbiguie/Desktop/Anomalie_eth/data/raw/transactions_20250115_204200.csv


Nombre de transactions collectées via Web3 : 17817


,hash,block_number,from_address,to_address,value,gas,block_timestamp,gas_price,transaction_fee
0,0xfe51bc8dbefc05d4049ec95d57faae442c41d2edb338...,21632038,0x576ee2ad743b6daDA280e88261Afb3e107BDAED9,0x3328F7f4A1D1C57c35df56bBf0c9dCAFCA309C49,0.100000,471312,1736970071,29.051617,1.369238e+07
1,0xa27eba8db4eb5dba74753315123127c8c28b1ba5e792...,21632038,0x41f7A9b39498f32adE0c8691866EE751b4b622EA,0x8934347DeE1DCd76D81eb38A6E3Ce7CaB5bd9c53,17.296497,21000,1736970071,24.200000,5.082000e+05
2,0xe93e09f5939982d965866dc6ccdd5bec3b8f1a01a02f...,21632038,0x7A6e99771575A5ac3D2796b7Ee0ad0Cce36a29Df,0x0eF56548dAF2eB21792F7d70D0A7D72c37F02683,0.001203,21000,1736970071,24.200000,5.082000e+05
3,0x264e837c8e614326dd750114a6e88538b207b48f989d...,21632038,0xa826e12fC46A4b2bff0107E6a3B787465f033F6c,0x5BF9A6E2859959689058872670bAcCda50533f38,0.029422,21000,1736970071,22.000000,4.620000e+05
4,0x10fe6c22bc08c330015696191f7949d5cca2cc1c868b...,21632038,0x9acbB72Cf67103A30333A32CD203459c6a9c3311,0xcf0C122c6b73ff809C693DB761e7BaeBe62b6a2E,0.000000,90000,1736970071,21.000000,1.890000e+06


## 3. Comparaison et Combinaison des Données

Analysons les différences entre les deux sources de données et créons un jeu de données unifié.

In [5]:
# Comparaison des colonnes
print("Colonnes Etherscan :", etherscan_transactions.columns.tolist())
print("\nColonnes Web3 :", web3_transactions.columns.tolist())

# Identification des transactions communes
common_txs = pd.merge(
    etherscan_transactions,
    web3_transactions,
    on='hash',
    how='inner',
    suffixes=('_etherscan', '_web3')
)

print(f"\nNombre de transactions communes : {len(common_txs)}")

Colonnes Etherscan : ['timestamp', 'block_number', 'hash', 'from_address', 'to_address', 'value', 'gas', 'gas_price']

Colonnes Web3 : ['hash', 'block_number', 'from_address', 'to_address', 'value', 'gas', 'block_timestamp', 'gas_price', 'transaction_fee']

Nombre de transactions communes : 152


In [6]:
# Sauvegarde des données combinées
output_file = RAW_DATA_DIR / f'combined_transactions_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
common_txs.to_csv(output_file, index=False)
print(f"Données sauvegardées dans : {output_file}")

Données sauvegardées dans : /Users/guillaumeverbiguie/Desktop/Anomalie_eth/data/raw/combined_transactions_20250115_204307.csv
